In [1]:
# import library
import pickle
import pandas as pd
import numpy as np

import xgboost as xgb
import lightgbm as lgb

from sklearn.ensemble import RandomForestClassifier as rf
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from keras import models
from keras import layers
from keras.layers.advanced_activations import LeakyReLU
from keras.utils import np_utils
from keras.layers import Dense, Dropout, Activation, Flatten, Input, Dense, GaussianNoise, GaussianDropout
from keras.models import Sequential, Model
from keras.regularizers import l2, l1
from keras.optimizers import Adam, SGD, Nadam
from keras.callbacks import LearningRateScheduler
from keras.metrics import categorical_accuracy
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasClassifier
from  keras.regularizers import l1, l2
from keras import backend as K
from keras.utils.generic_utils import get_custom_objects

import matplotlib.pyplot as plt

%matplotlib inline

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto(
    gpu_options=tf.GPUOptions(
        visible_device_list="1", 
        allow_growth=True,
#         per_process_gpu_memory_fraction=0.5
    )
)
set_session(tf.Session(config=config))

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# read data
df = pd.read_csv('../data/decomp_pos.csv')


# divide objective and target
objective = df.Subclass
le = preprocessing.LabelEncoder()
objective = le.fit_transform(objective)
features = df.drop('Subclass', axis=1)

# train test split
random_state=np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(
    features, 
    objective,
    test_size=0.2
)

# transform  for keras's target label
y_train_for_keras = np_utils.to_categorical(y_train)
y_test_for_keras = np_utils.to_categorical(y_test)

In [3]:
# define and fit
clf = rf()
clf = clf.fit(X_train, y_train)

# result append to list
t = []
t.append(round(clf.score(X_test, y_test)*100, 2))
# pickle.dump(clf, open('../model/rf_pos_fs.sav', 'wb'))

In [4]:
# define and fit
gbm = lgb.LGBMClassifier(
    objective='multiclass',
    device = 'gpu'
)

gbm.fit(
    X_train, 
    y_train,
    eval_set=[(X_test, y_test)],
    early_stopping_rounds=5
)

# pickle.dump(gbm, open('../model/lgbm_pos_fs.sav', 'wb'))
# result append to list
t.append(round(gbm.score(X_test, y_test)*100, 2))

[1]	valid_0's multi_logloss: 1.29038
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's multi_logloss: 1.2103
[3]	valid_0's multi_logloss: 1.14562
[4]	valid_0's multi_logloss: 1.08966
[5]	valid_0's multi_logloss: 1.04154
[6]	valid_0's multi_logloss: 0.99875
[7]	valid_0's multi_logloss: 0.960067
[8]	valid_0's multi_logloss: 0.926992
[9]	valid_0's multi_logloss: 0.894829
[10]	valid_0's multi_logloss: 0.863366
[11]	valid_0's multi_logloss: 0.83366
[12]	valid_0's multi_logloss: 0.810582
[13]	valid_0's multi_logloss: 0.784551
[14]	valid_0's multi_logloss: 0.767014
[15]	valid_0's multi_logloss: 0.742244
[16]	valid_0's multi_logloss: 0.723661
[17]	valid_0's multi_logloss: 0.707815
[18]	valid_0's multi_logloss: 0.690551
[19]	valid_0's multi_logloss: 0.673864
[20]	valid_0's multi_logloss: 0.658861
[21]	valid_0's multi_logloss: 0.645815
[22]	valid_0's multi_logloss: 0.632745
[23]	valid_0's multi_logloss: 0.621788
[24]	valid_0's multi_logloss: 0.611439
[25]	valid_0's multi

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [5]:
# set parameters
params = {}
params['device'] = 'gpu'
params['gpu_id'] = 1
params['updater'] = 'grow_gpu_hist'

In [6]:
# define and fit
model = xgb.XGBClassifier(**params)
model.fit(X_train, y_train)

# result append to list
# pickle.dump(model, open('../model/xgb_pos_fs.sav', 'wb'))
t.append(round(model.score(X_test, y_test)*100, 2))

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [7]:
tf.set_random_seed(42)

# make keras model

inputs = Input(shape=(X_train.shape[1],))

x = Dense(512, activation='relu')(inputs)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

predictions = Dense(len(df['Subclass'].value_counts()), activation='softmax')(x)
model = Model(inputs=inputs, outputs=predictions)

# compile
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(),
    metrics=['accuracy']
)

# fit
epochs = 100
batch_size = 1000
es = EarlyStopping(monitor='val_loss', patience=20)

history = model.fit(
    X_train,
    y_train_for_keras,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(X_test, y_test_for_keras),
    verbose=1,
    callbacks=[
        es,
#         lr_decay
    ]
)

# model.save('../model/Keras_pos_fs.h5')
t.append(round(model.evaluate(X_test, y_test_for_keras, verbose=0)[1]*100, 2))

Train on 1340 samples, validate on 336 samples
Epoch 1/100
1340/1340 [==============================] - 3s 2ms/step - loss: 1.9215 - acc: 0.2366 - val_loss: 1.4388 - val_acc: 0.2440
Epoch 2/100
1340/1340 [==============================] - 1s 722us/step - loss: 1.8227 - acc: 0.2664 - val_loss: 1.4346 - val_acc: 0.2649
Epoch 3/100
1340/1340 [==============================] - 1s 731us/step - loss: 1.6891 - acc: 0.2940 - val_loss: 1.4262 - val_acc: 0.3512
Epoch 4/100
1340/1340 [==============================] - 1s 731us/step - loss: 1.6033 - acc: 0.2970 - val_loss: 1.4055 - val_acc: 0.4077
Epoch 5/100
1340/1340 [==============================] - 1s 728us/step - loss: 1.5583 - acc: 0.3239 - val_loss: 1.3145 - val_acc: 0.5060
Epoch 6/100
1340/1340 [==============================] - 1s 747us/step - loss: 1.5217 - acc: 0.3313 - val_loss: 1.2889 - val_acc: 0.4137
Epoch 7/100
1340/1340 [==============================] - 1s 776us/step - loss: 1.5359 - acc: 0.3515 - val_loss: 1.2818 - val_acc: 0.4

In [8]:
f = []
f.append(t)

In [9]:
f

[[74.4, 83.63, 80.65, 70.54]]

In [10]:
#read data
df = pd.read_csv('../data/feature_selection_negative.csv')

# divide objective and target
objective = df.Subclass
le = preprocessing.LabelEncoder()
objective = le.fit_transform(objective)
features = df.drop('Subclass', axis=1)

# train test split
random_state=np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(
    features, 
    objective,
    test_size=0.2
)

# transform  for keras's target label
y_train_for_keras = np_utils.to_categorical(y_train)
y_test_for_keras = np_utils.to_categorical(y_test)

In [11]:
# define and fit 
clf = rf()
clf = clf.fit(X_train, y_train)

# result append to list
t = []
t.append(round(clf.score(X_test, y_test)*100, 2))
# pickle.dump(clf, open('../model/rf_ng_fs.sav', 'wb'))

In [12]:
# define and fit
gbm = lgb.LGBMClassifier(
    objective='multiclass',
    device = 'gpu'
)

gbm.fit(
    X_train, 
    y_train,
    eval_set=[(X_test, y_test)],
    early_stopping_rounds=5
)

# pickle.dump(gbm, open('../model/lgbm_ng_fs.sav', 'wb'))
# result append to list
t.append(round(gbm.score(X_test, y_test)*100, 2))

[1]	valid_0's multi_logloss: 1.30732
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's multi_logloss: 1.24253
[3]	valid_0's multi_logloss: 1.17804
[4]	valid_0's multi_logloss: 1.13119
[5]	valid_0's multi_logloss: 1.0872
[6]	valid_0's multi_logloss: 1.04603
[7]	valid_0's multi_logloss: 1.00813
[8]	valid_0's multi_logloss: 0.973739
[9]	valid_0's multi_logloss: 0.944839
[10]	valid_0's multi_logloss: 0.914158
[11]	valid_0's multi_logloss: 0.890913
[12]	valid_0's multi_logloss: 0.867857
[13]	valid_0's multi_logloss: 0.849797
[14]	valid_0's multi_logloss: 0.831871
[15]	valid_0's multi_logloss: 0.820567
[16]	valid_0's multi_logloss: 0.803571
[17]	valid_0's multi_logloss: 0.791859
[18]	valid_0's multi_logloss: 0.77767
[19]	valid_0's multi_logloss: 0.769352
[20]	valid_0's multi_logloss: 0.759553
[21]	valid_0's multi_logloss: 0.751557
[22]	valid_0's multi_logloss: 0.744757
[23]	valid_0's multi_logloss: 0.735332
[24]	valid_0's multi_logloss: 0.727783
[25]	valid_0's multi_

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [13]:
# setting parameters
params = {}
params['device'] = 'gpu'
params['gpu_id'] = 1
params['updater'] = 'grow_gpu_hist'

In [14]:
# define and fit 
model = xgb.XGBClassifier(**params)
model.fit(X_train, y_train)

# result append to list
t.append(round(model.score(X_test, y_test)*100, 2))
# pickle.dump(model, open('../model/xgb_ng_fs.sav', 'wb'))

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [15]:
tf.set_random_seed(42)

# make keras model

inputs = Input(shape=(X_train.shape[1],))

x = Dense(512, activation='relu')(inputs)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

predictions = Dense(len(df['Subclass'].value_counts()), activation='softmax')(x)
model = Model(inputs=inputs, outputs=predictions)

# compile
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(),
    metrics=['accuracy']
)

# fit
epochs = 100
batch_size = 1000
es = EarlyStopping(monitor='val_loss', patience=20)

history = model.fit(
    X_train,
    y_train_for_keras,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(X_test, y_test_for_keras),
    verbose=1,
    callbacks=[
        es,
#         lr_decay
    ]
)

# model.save('../model/Keras_ng_fs.h5')
t.append(round(model.evaluate(X_test, y_test_for_keras, verbose=0)[1]*100, 2))

Train on 373 samples, validate on 94 samples
Epoch 1/100
373/373 [==============================] - 1s 4ms/step - loss: 2.2138 - acc: 0.2064 - val_loss: 1.3822 - val_acc: 0.3723
Epoch 2/100
373/373 [==============================] - 0s 56us/step - loss: 1.9182 - acc: 0.2761 - val_loss: 1.3613 - val_acc: 0.4681
Epoch 3/100
373/373 [==============================] - 0s 60us/step - loss: 1.8160 - acc: 0.3164 - val_loss: 1.4258 - val_acc: 0.5106
Epoch 4/100
373/373 [==============================] - 0s 57us/step - loss: 1.6460 - acc: 0.3887 - val_loss: 1.5049 - val_acc: 0.5000
Epoch 5/100
373/373 [==============================] - 0s 81us/step - loss: 1.6544 - acc: 0.3968 - val_loss: 1.5190 - val_acc: 0.5000
Epoch 6/100
373/373 [==============================] - 0s 57us/step - loss: 1.6293 - acc: 0.3914 - val_loss: 1.4847 - val_acc: 0.4894
Epoch 7/100
373/373 [==============================] - 0s 57us/step - loss: 1.5146 - acc: 0.3619 - val_loss: 1.4169 - val_acc: 0.5106
Epoch 8/100
373/37

Epoch 62/100
373/373 [==============================] - 0s 47us/step - loss: 0.6653 - acc: 0.6917 - val_loss: 0.8776 - val_acc: 0.6277
Epoch 63/100
373/373 [==============================] - 0s 49us/step - loss: 0.6872 - acc: 0.7078 - val_loss: 0.8776 - val_acc: 0.6383
Epoch 64/100
373/373 [==============================] - 0s 49us/step - loss: 0.6412 - acc: 0.7292 - val_loss: 0.8746 - val_acc: 0.6383
Epoch 65/100
373/373 [==============================] - 0s 49us/step - loss: 0.7212 - acc: 0.6810 - val_loss: 0.8740 - val_acc: 0.6383
Epoch 66/100
373/373 [==============================] - 0s 50us/step - loss: 0.6239 - acc: 0.7373 - val_loss: 0.8757 - val_acc: 0.6383
Epoch 67/100
373/373 [==============================] - 0s 52us/step - loss: 0.6434 - acc: 0.7212 - val_loss: 0.8756 - val_acc: 0.6383
Epoch 68/100
373/373 [==============================] - 0s 47us/step - loss: 0.6567 - acc: 0.7292 - val_loss: 0.8772 - val_acc: 0.6383
Epoch 69/100
373/373 [==============================] -

In [16]:
f.append(t)

In [17]:
# save to csv
pd.DataFrame(
    f, 
    columns=['Random Forest', 'LightGBM', 'XGBoost', 'Keras'],
    index=['Positive', 'Negative']
)
# .to_csv('../result/feature_selection.csv')

,Random Forest,LightGBM,XGBoost,Keras
Positive,74.40,83.63,80.65,70.54
Negative,72.34,74.47,74.47,64.89
